In [1]:
%load_ext autoreload
%autoreload 2

import os, json
import nibabel as nb

from keras import metrics 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LocallyConnected1D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from keras.utils.vis_utils import plot_model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

base_dir = '/home/knapen/projects/prf_lyon/'
os.chdir(base_dir)

os.chdir(os.path.join(base_dir, 'analysis'))
from prf_fit import *

with open(os.path.join(base_dir, 'analysis', 'settings.json')) as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)
    

Using TensorFlow backend.
/home/knapen/software/miniconda3/envs/py36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
sub = 'sub-01'

input_file = os.path.join(base_dir, 'data', sub, 'func', \
    sub+'_task-prf_acq-median_T1w_desc-preproc_bold.nii.gz')
dm_file = os.path.join(base_dir, 'data', sub, 'dm_out.npy')
    
mask_file = nb.load(os.path.join(base_dir, 'data', sub, 'func', \
     sub+'_task-prf_dir-AP_run-1_space-T1w_desc-brain_mask.nii.gz'))
mask = mask_file.get_data().astype(bool)

# for registration into pycortex
example_epi_file = os.path.join(base_dir, 'data', sub, 'func', \
     sub+'_task-prf_dir-AP_run-1_space-T1w_boldref.nii.gz')
T1_file = os.path.join(base_dir, 'data', sub, 'anat', \
     sub+'_desc-preproc_T1w.nii.gz')
fs_T1_file = os.path.join(base_dir, 'data', sub, 'anat', \
     'T1.nii.gz')

#design matrix
visual_dm = np.load(dm_file).T

# data
in_file_nii = nb.load(input_file)
data = in_file_nii.get_data().reshape((-1,in_file_nii.shape[-1]))

In [3]:
fit_model = analysis_info["fit_model"]
N_PROCS = 8

# Fit: define search grids
x_grid_bound = (-analysis_info["max_eccen"], analysis_info["max_eccen"])
y_grid_bound = (-analysis_info["max_eccen"], analysis_info["max_eccen"])
sigma_grid_bound = (analysis_info["min_size"], analysis_info["max_size"])
n_grid_bound = (analysis_info["min_n"], analysis_info["max_n"])
grid_steps = analysis_info["grid_steps"]

# Fit: define search bounds
x_fit_bound = (-analysis_info["max_eccen"]*2, analysis_info["max_eccen"]*2)
y_fit_bound = (-analysis_info["max_eccen"]*2, analysis_info["max_eccen"]*2)
sigma_fit_bound = (1e-6, 1e2)
n_fit_bound = (1e-6, 2)
beta_fit_bound = (-1e6, 1e6)
baseline_fit_bound = (-1e6, 1e6)

if fit_model == 'gauss' or fit_model == 'gauss_sg':
    bound_grids  = (x_grid_bound, y_grid_bound, sigma_grid_bound)
    bound_fits = (x_fit_bound, y_fit_bound, sigma_fit_bound, beta_fit_bound, baseline_fit_bound)
elif fit_model == 'css' or fit_model == 'css_sg':
    bound_grids  = (x_grid_bound, y_grid_bound, sigma_grid_bound, n_grid_bound)
    bound_fits = (x_fit_bound, y_fit_bound, sigma_fit_bound, n_fit_bound, beta_fit_bound, baseline_fit_bound)

# intitialize prf analysis
prf = PRF_fit(  data = data[mask.ravel()],
                fit_model = fit_model, 
                visual_design = visual_dm, 
                screen_distance = analysis_info["screen_distance"],
                screen_width = analysis_info["screen_width"],
                scale_factor = 1/2.0, 
                tr =  analysis_info["TR"],
                bound_grids = bound_grids,
                grid_steps = grid_steps,
                bound_fits = bound_fits,
                n_jobs = N_PROCS,
                sg_filter_window_length = analysis_info["sg_filt_window_length"],
                sg_filter_polyorder = analysis_info["sg_filt_polyorder"],
                sg_filter_deriv = analysis_info["sg_filt_deriv"], 
                )
# will need to move/delete this file for new predictions
prediction_file = os.path.join(base_dir, 'data', 'sub-01', 'predictions.npy')
if os.path.isfile(prediction_file):
    prf.load_grid_predictions(prediction_file=prediction_file)
else:
    prf.make_predictions(out_file=prediction_file)

In [31]:
blow_up = 100
# make extra predictions, with added noise
blow_up_noisy_predictions = np.repeat(prf.predictions, blow_up, axis=1)
# scale predictions randomly
betas = np.random.rand(blow_up_noisy_predictions.shape[1])+0.25
blow_up_noisy_predictions *= betas
# random offsets
offsets = np.random.randn(blow_up_noisy_predictions.shape[1])
blow_up_noisy_predictions += offsets

blow_up_noisy_predictions += np.random.randn(prf.predictions.shape[0], prf.predictions.shape[1] * blow_up)/5.0

extra_parameters = np.array([betas, offsets])

n_prf_parameters = 5
parameters = np.array([prf.prf_xs, prf.prf_ys, prf.prf_sigma]).reshape((3,-1))

blow_up_parameters = np.vstack((np.repeat(parameters, blow_up, axis=1), extra_parameters))

In [32]:
# build model
model = Sequential()

model.add(Conv1D(filters=prf.n_timepoints//2, kernel_size=4, input_shape=(prf.n_timepoints,1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=prf.n_timepoints//4, kernel_size=4, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LocallyConnected1D(filters=prf.n_timepoints//8, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LocallyConnected1D(filters=prf.n_timepoints//16, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
# model.add(Conv1D(filters=prf.n_timepoints//32, kernel_size=16, activation='relu'))
# model.add(MaxPooling1D(pool_size=4))
model.add(Flatten())
model.add(Dense(n_prf_parameters*2, kernel_initializer='uniform'))
model.add(Dense(n_prf_parameters))

print(model.summary())
plot_model(model, show_shapes=True, to_file='keras_1.pdf')

model.compile(loss='mse',
              optimizer='nadam',
              metrics=['mae'])
# model.fit(prf.predictions.T[:,:,np.newaxis], parameters.T, epochs=10, batch_size=32, verbose=3)
model.fit(blow_up_noisy_predictions.T[:,:,np.newaxis], blow_up_parameters.T, 
          epochs=10, 
          batch_size=256, 
          verbose=3)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 164, 83)           415       
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 82, 83)            0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 79, 41)            13653     
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, 39, 41)            0         
_________________________________________________________________
locally_connected1d_7 (Local (None, 32, 20)            210560    
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, 16, 20)            0         
_________________________________________________________________
locally_connected1d_8 (Local (None, 9, 10)             14490     
__________

In [33]:
output_pars = model.predict(prf.data[:,:,np.newaxis])

In [34]:
parameters = np.array([prf.prf_xs, prf.prf_ys, prf.prf_sigma, np.ones_like(prf.prf_sigma), np.zeros_like(prf.prf_sigma) ]).reshape((5,-1))
pred = model.predict(prf.predictions.T[:,:,np.newaxis])

print(sp.stats.pearsonr(pred.ravel(), parameters.T[:].ravel()))

(0.99833249407842617, 0.0)


In [21]:
model.predict(prf.predictions.T)


(3, 8000)